In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "MyProgram")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=MyProgram, master=local) created by __init__ at <ipython-input-1-10be1c77ae47>:2 

In [8]:
clientes = sc.textFile("datasets/Banco/Clientes.txt")
clientes = clientes.map(lambda line: line.split("\t"))
#clientes = clientes.map(lambda t : (int(t[0]), t[1] + " " + t[2], int(t[3]), t[4], t[5]))

In [9]:
# El banco tiene más clientes europeos que americanos
paises = clientes.map(lambda t: t[5]).distinct()

total = clientes.count()
americanos = clientes.filter(lambda t: t[5] in ['BRA', 'ARG', 'VEN', 'COL', 'BOL', 'ECU', 'URU', 'PAR', 'CHI', 'PER'])
total_americanos = americanos.count()
europeos = clientes.filter(lambda t: t[5] in ['ITA', 'ESP'])
total_europeos = total - total_americanos

print("Total de clientes: " + str(total))
print("Cantidad de clientes americanos: "+ str(total_americanos)) 
print("Cantidad de clientes europeos: "+ str(total_europeos)) 

print("El banco tiene más clientes europeos que americanos: " + str(total_europeos > total_americanos))

Total de clientes: 27087
Cantidad de clientes americanos: 22662
Cantidad de clientes europeos: 4425
El banco tiene más clientes europeos que americanos: False


In [10]:
# b) El promedio de edad de los clientes americanos es menor que el de los europeos
from datetime import datetime

a = americanos.map(lambda t: (datetime.now() - datetime.strptime(t[4], '%Y-%m-%d')).days // 365)

e = europeos.map(lambda t: (datetime.now() - datetime.strptime(t[4], '%Y-%m-%d')).days // 365)

a = a.reduce(lambda t1, t2: (t1 + t2)/2)
e = e.reduce(lambda t1, t2: (t1 + t2)/2)

print("Promedio de edad de los clientes europeos: "+ str(e))
print("Promedio de edad de los clientes americanos: "+ str(a)) 

print("El promedio de edad de los clientes americanos es menor que el de los europeos: " + str(a < e))

Promedio de edad de los clientes europeos: 47.41717062334077
Promedio de edad de los clientes americanos: 46.59238163019565
El promedio de edad de los clientes americanos es menor que el de los europeos: True


In [12]:
#c) Los americanos deben más plata que los europeos (un cliente debe plata si la suma de montos de todas sus cajas de ahorro es negativa)

# <id_caja>, <id_cliente>, <monto>
cajas = sc.textFile("datasets/Banco/CajasDeAhorro.txt")
cajas = cajas.map(lambda line: line.split("\t"))
c = cajas.map(lambda t: (t[1], t[2]))

# <id_cliente>, <monto>
c = c.reduceByKey(lambda t1, t2: (float(t1) + float(t2)))

#def func(t1,t2):
#americanos_k = americanos.map(lambda t1: (t1[0],(t1[1:])))

montos_americanos = americanos.join(c)
montos_americanos = montos_americanos.map(lambda t: (t[1][1]))

montos_europeos = europeos.join(c)
montos_europeos = montos_europeos.map(lambda t: (t[1][1]) )

montos_americanos = montos_americanos.reduce(lambda t1, t2: (float(t1) + float(t2)))
montos_europeos = montos_europeos.reduce(lambda t1, t2: (float(t1) + float(t2))) 

print("Los americanos deben más plata que los europeos: " + str(montos_americanos < montos_europeos))

Los americanos deben más plata que los europeos: True


In [17]:
# d) Los clientes americanos suelen sacar, en promedio, préstamos con mayor cantidad de cuotas que los europeos

# <id_caja>, <monto>, <cuotas>
prestamos = sc.textFile("datasets/Banco/Prestamos.txt")
prestamos = prestamos.map(lambda line: line.split("\t"))

# <id_caja>, <cuotas>
p = prestamos.map(lambda t: (t[0],t[2]))
prestamos_sum = p.reduceByKey(lambda t1, t2: (float(t1[1]) + float(t2[1])))

#<id_cliente>, <Nombre>, <Apellido>, <DNI>, <Nacionalidad>, <FechaNacimiento>
#c = clientes.map(lambda t: t)

# <id_caja>, <id_cliente>, <monto>
# <id_cliente>, <id_caja>
c = cajas.map(lambda t: (t[1], t[0]))

# <id_cliente>, <id_caja>
americanos_caja = americanos.join(c)

# <id_caja>,<id_cliente>
americanos_caja = americanos_caja.map(lambda t: (t[1][1],t[0]))
aux = americanos_caja.join(prestamos_sum)

print(c.first())
print(americanos.first())
print(americanos_caja.first())

#americanos_caja = americanos_caja.map(lambda t: t[0])


('1', '1')
['1', 'juan', 'topo', '6', '1991-05-20', 'ARG']
('1', '1')


In [15]:
print(prestamos_sum.first())
print(americanos.first())
print(americanos_caja.take(3))
print(p.first())
print(aux.first())

('1', '90')
['1', 'juan', 'topo', '6', '1991-05-20', 'ARG']
[('1', '1'), ('1', '1'), ('760246', '39670')]
('1', '90')
('836426', ('46037', '8192.30250436454'))
